### DESCRIPTION

Run S2A systems technoeconomic analysis. Run "s2a_systems_functions.py" for user-specified input parameters.

Outputs:
- "input params.csv": all sets of input parameters mapped to run #.
- "input params_varied.csv": input parameters that are different from baseline input parameters (run 0) mapped to run #.
- "output_[xxxx].csv": input parameters and results for each run. Filename suffix = run #.
- "output_all runs.csv": input parameters and results for all runs.
- "output_all runs_refueling station costs.csv": refueling station inputs and costs for "LOHC - formic acid" pathway for all runs.
- "output_all runs_refueling station costs_pivot.csv": refueling station inputs and costs for "LOHC - formic acid" pathway for all runs, pivot format.
- "output_cost by [pathway/process/location/function/process-function].csv": input parameters and cost results for all runs aggregated by pathway, process (e.g., preconditioning), location (e.g., terminal), function (e.g., compression), or by process and function.
- "output_emissions by [pathway/process/location/function/process-function].csv": input parameters and emissions results for all runs aggregated by pathway, process (e.g., preconditioning), location (e.g., terminal), function (e.g., compression), or by process and function.

### IMPORT MODULES

In [1]:
import s2a_systems_functions as s2a_sys
import numpy as np
import pandas as pd
import os
from datetime import date

In [2]:
# show default arguments for reference
help(s2a_sys.calcs)

Help on function calcs in module s2a_systems_functions:

calcs(dict_input_params={'run #': 0, 'output dollar year': 2022, 'target station capacity (kg/day)': 1000.0, 'station annual capacity factor': 1.0, 'target number of stations': 10, 'one-way delivery distance (mile)': 100.0, 'electricity cost ($/kWh)': 0.1709, 'diesel cost ($/gallon)': 6.028, 'electricity emission factor (kg CO2/kWh)': 0.228, 'diesel emission factor (kg CO2/gallon)': 10.18, 'hydrogen prod. emission factor (kg CO2-eq/kg)': 0.0, 'LOHC prod. emission factor (kg CO2-eq/kg)': 0.0, 'hydrogen purchase cost ($/kg)': 0.31, 'LOHC purchase cost ($/kg)': 1.0, 'LOHC name': 'formic acid', 'LOHC molar mass (kg/kmol)': 46.025, 'LOHC density (kg/m^3)': 1220.0, 'stoic. ratio (mol H2/mol LOHC)': 1, 'stoic. ratio (mol CO2/mol LOHC)': 1, 'stoic. ratio (mol e/mol LOHC)': 2, 'LOHC production pathway': 'electro', 'hydr. reaction temperature (K)': 366.15, 'hydr. reaction pressure (bar)': 105.0, 'hydr. reaction yield': 1.0, 'hydr. reactor 

### USER INPUT

In [3]:
# specify input parameter file
df_input_params_all = pd.read_excel('inputs/input params_baseline.xlsx')

# specify output folder
output_folder = 'outputs/outputs ' + date.today().strftime("%Y-%m-%d") + ' baseline'

### RUN SYSTEMS ANALYSIS

In [4]:
# create output folder if not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [5]:
# fill NaN values in input parameter file
df_input_params_all.ffill(inplace = True)

df_input_params_all

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,dehydr. reaction pressure (bar),dehydr. reaction yield,dehydr. reactor volume (m^3),number of dehydr. reactors,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. reactor energy (unit TBD),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,0.9999,0.073103,1,9.646712,3500,1,0,0,1
1,FA - delivery only,1,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,1,0.9999,0.073103,1,9.646712,3500,1,0,0,1
2,FA - closed loop,2,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,1,0.9999,0.073103,1,9.646712,3500,1,0,0,1
3,FA - closed loop - 80 cap factor,3,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,1,0.9999,0.073103,1,9.646712,3500,1,0,0,1


In [6]:
# create output dataframe for all runs
df_output_all = pd.DataFrame()

# run systems analysis for each set of input parameters
for i in df_input_params_all.index:
    
    # create dictionary of input parameters for each run
    df_input_params = df_input_params_all.iloc[i]
    dict_input_params = df_input_params.to_dict()

    # run systems analysis
    df_output, _, _, _ = s2a_sys.calcs(
        dict_input_params, 
        save_csv = True,
        output_folder = output_folder
        )

    # add run number column and rearrange columns
    cols = df_output.columns.tolist()
    df_output['run #'] = df_input_params['run #']
    df_output = df_output[['run #'] + cols]

    # add output dataframe to dataframe for all runs
    df_output_all = pd.concat([df_output_all, df_output])

In [7]:
# save output dataframe for all runs as csv
df_output_all.to_csv(
    os.path.join(output_folder, 'output_all runs.csv'),         
    index = False
    )

df_output_all

,run #,pathway,process,location,function,equipment,variable group,variable name,unit,value
0,0,all,all,all,all,all,input parameter,scenario,-,baseline
1,0,all,all,all,all,all,input parameter,run #,-,0
2,0,all,all,all,all,all,input parameter,output dollar year,-,2022
3,0,all,all,all,all,all,input parameter,target station capacity,kg/day,1000
4,0,all,all,all,all,all,input parameter,station annual capacity factor,-,1.0
...,...,...,...,...,...,...,...,...,...,...
430,3,LOHC - formic acid,reconditioning,refueling station,storage,liquid storage,capital cost,levelized capital cost,$/kg H2,0.023007
431,3,LOHC - formic acid,reconditioning,refueling station,storage,cascade storage,capital cost,levelized capital cost,$/yr,1202355.235363
432,3,LOHC - formic acid,reconditioning,refueling station,storage,cascade storage,capital cost,levelized capital cost,$/kg H2,0.411765
433,3,LOHC - formic acid,reconditioning,refueling station,reaction,reactor,emissions,process emissions,kg CO2/kg H2,0.0


### PROCESS OUTPUT

In [8]:
# drop scenario and run number rows
df_output_all = df_output_all.loc[
    ~df_output_all['variable name'].isin(['scenario', 'run #'])
    ].reset_index(drop = True)

In [9]:
# filter for $/kg H2 cost results
df_usd_per_kg = df_output_all.loc[
    df_output_all['unit'] == '$/kg H2'
    ].reset_index(drop = True)

# filter for kg CO2/kg H2 emissions results
df_kgCO2_per_kg = df_output_all.loc[
    df_output_all['unit'] == 'kg CO2/kg H2'
    ].reset_index(drop = True)

#### OUTPUT: all input parameters

In [10]:
# write input parameters as csv to output folder
df_input_params_all.to_csv(
    os.path.join(output_folder, 'input params.csv'),         
    index = False
    )

#### OUTPUT: varied input parameters
including base and user-defined values 

In [11]:
# find column names of varied input parameters
varied_cols = []

for col in df_input_params_all.columns[2:]:
    if any(df_input_params_all[col] != df_input_params_all[col][0]):
        varied_cols.append(col)

# extract varied input parameters
df_input_params_varied = df_input_params_all[
    ['scenario', 'run #'] + varied_cols
    ]

# save dataframe as csv
df_input_params_varied.to_csv(
    os.path.join(output_folder, 'input params_varied.csv'),         
    index = False
    )

df_input_params_varied

,scenario,run #,station annual capacity factor,electricity emission factor (kg CO2/kWh),hydrogen purchase cost ($/kg),LOHC production pathway
0,baseline,0,1.0,0.228,0.31,electro
1,FA - delivery only,1,1.0,0.000,0.00,purchase
2,FA - closed loop,2,1.0,0.000,0.31,electro
3,FA - closed loop - 80 cap factor,3,0.8,0.000,0.31,electro


#### OUTPUT: refueling station costs for process modeling team

In [12]:
# set filters for refueling station costs
# including catalyst amount and reactor volume
pathway = ['LOHC - formic acid', 'all']
location = ['refueling station', 'all']
units = ['$/yr', '$/kg H2', '$/station', 'kg', 'm^3']
# equipment = [
#     'compressor',
#     'refrigerator',
#     'cryogenic pump',
#     'reactor pump',
#     'reactor',
#     'catalyst',
#     'PSA refrigerator',
#     'PSA'
#     ]

# filter results
df_stn = df_output_all.loc[
    df_output_all['pathway'].isin(pathway) & \
    df_output_all['location'].isin(location) & \
    df_output_all['unit'].isin(units)
    ].reset_index(drop = True)

# filter out hydrogenation inputs
df_stn = df_stn.loc[
    ~df_stn['variable name'].str.startswith('hydr.')
    ]

# save dataframe as csv
df_stn.to_csv(
    os.path.join(
        output_folder, 
        'output_all runs_refueling station costs.csv'
        ),         
    index = False
    )

df_stn

,run #,pathway,process,location,function,equipment,variable group,variable name,unit,value
2,0,all,all,all,all,all,input parameter,dehydr. reactor volume,m^3,0.073103
3,0,all,all,all,all,all,input parameter,dehydr. catalyst amount,kg,9.646712
4,0,LOHC - formic acid,reconditioning,refueling station,pumping,reactor pump,energy cost,electricity cost,$/yr,0.0
5,0,LOHC - formic acid,reconditioning,refueling station,pumping,reactor pump,energy cost,electricity cost,$/kg H2,0.0
6,0,LOHC - formic acid,reconditioning,refueling station,pumping,reactor pump,O&M cost,"operation, maintenance, repair costs",$/yr,0.0
...,...,...,...,...,...,...,...,...,...,...
339,3,LOHC - formic acid,reconditioning,refueling station,cooling,refrigerator,capital cost,levelized capital cost,$/kg H2,0.10983
340,3,LOHC - formic acid,reconditioning,refueling station,storage,liquid storage,capital cost,levelized capital cost,$/yr,67181.346462
341,3,LOHC - formic acid,reconditioning,refueling station,storage,liquid storage,capital cost,levelized capital cost,$/kg H2,0.023007
342,3,LOHC - formic acid,reconditioning,refueling station,storage,cascade storage,capital cost,levelized capital cost,$/yr,1202355.235363


In [13]:
# create column of output name
df_stn['name'] = \
    df_stn['equipment'] + ' ' + \
    df_stn['variable name'] + ' (' + \
    df_stn['unit'] + ')'

# drop unused columns
df_stn = df_stn[[
    'run #', 
    'name',
    'value',
    ]]

# pivot dataframe
df_stn_pivot = df_stn.pivot(
    columns = 'name', 
    index = 'run #', 
    values = 'value'
    ).reset_index()

# remove name of columns
df_stn_pivot.columns.name = None

# merge in varied input parameters
df_stn_pivot = df_input_params_varied.merge(
    right = df_stn_pivot, 
    on='run #', 
    how = 'right'
    )

# save dataframe as csv
df_stn_pivot.to_csv(
    os.path.join(
        output_folder, 
        'output_all runs_refueling station costs_pivot.csv'
        ),         
    index = False
    )

df_stn_pivot

,scenario,run #,station annual capacity factor,electricity emission factor (kg CO2/kWh),hydrogen purchase cost ($/kg),LOHC production pathway,PSA electricity cost ($/kg H2),PSA electricity cost ($/yr),PSA labor cost ($/kg H2),PSA labor cost ($/yr),...,refrigerator electricity cost ($/kg H2),refrigerator electricity cost ($/yr),refrigerator labor cost ($/kg H2),refrigerator labor cost ($/yr),refrigerator levelized capital cost ($/kg H2),refrigerator levelized capital cost ($/yr),"refrigerator operation, maintenance, repair costs ($/kg H2)","refrigerator operation, maintenance, repair costs ($/yr)",refrigerator other O&M costs ($/kg H2),refrigerator other O&M costs ($/yr)
0,baseline,0,1.0,0.228,0.31,electro,1.01959,3721504.976895,0.063001,229954.948572,...,0.05985,218451.759177,0.004197,15317.98512,0.087864,320703.803737,0.009923,36218.114993,0.012815,46775.695513
1,FA - delivery only,1,1.0,0.000,0.00,purchase,1.01959,3721504.976895,0.063001,229954.948572,...,0.05985,218451.759177,0.004197,15317.98512,0.087864,320703.803737,0.009923,36218.114993,0.012815,46775.695513
2,FA - closed loop,2,1.0,0.000,0.31,electro,1.01959,3721504.976895,0.063001,229954.948572,...,0.05985,218451.759177,0.004197,15317.98512,0.087864,320703.803737,0.009923,36218.114993,0.012815,46775.695513
3,FA - closed loop - 80 cap factor,3,0.8,0.000,0.31,electro,1.01959,2977203.981516,0.078752,229954.948572,...,0.05985,174761.407341,0.005246,15317.98512,0.10983,320703.803737,0.012403,36218.114993,0.016019,46775.695513


#### OUTPUT: total costs ($/kg H2) and emissions (kg CO2/kg H2) by pathway

In [14]:
# calculate total $/kg H2 cost by pathway
df_usd_per_kg_path = df_usd_per_kg.groupby(
    by = ['run #', 'pathway'])['value'].sum().reset_index()

# update column name
df_usd_per_kg_path.rename(
    columns = {'value' : 'total levelized cost of hydrogen ($/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_usd_per_kg_path = df_input_params_all.merge(
    right = df_usd_per_kg_path, 
    on = 'run #'
    )

# save dataframe as csv
df_usd_per_kg_path.to_csv(
    os.path.join(output_folder, 'output_cost by pathway.csv'),         
    index = False
    )

df_usd_per_kg_path

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,dehydr. reactor volume (m^3),number of dehydr. reactors,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. reactor energy (unit TBD),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days),pathway,total levelized cost of hydrogen ($/kg H2)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,0.073103,1,9.646712,3500,1,0,0,1,LOHC - formic acid,32.243546
1,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,0.073103,1,9.646712,3500,1,0,0,1,compressed hydrogen,6.171953
2,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,0.073103,1,9.646712,3500,1,0,0,1,liquid hydrogen,7.273821
3,FA - delivery only,1,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500,1,0,0,1,LOHC - formic acid,7.245371
4,FA - delivery only,1,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500,1,0,0,1,compressed hydrogen,5.861953
5,FA - delivery only,1,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500,1,0,0,1,liquid hydrogen,6.963821
6,FA - closed loop,2,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500,1,0,0,1,LOHC - formic acid,32.243546
7,FA - closed loop,2,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500,1,0,0,1,compressed hydrogen,6.171953
8,FA - closed loop,2,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500,1,0,0,1,liquid hydrogen,7.273821
9,FA - closed loop - 80 cap factor,3,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500,1,0,0,1,LOHC - formic acid,36.477827


In [15]:
# calculate total kg CO2/kg H2 emissions by pathway
df_kgCO2_per_kg_path = df_kgCO2_per_kg.groupby(
    by = ['run #', 'pathway'])['value'].sum().reset_index()

# update column name
df_kgCO2_per_kg_path.rename(
    columns = {'value' : 'total emissions (kg CO2-eq/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_kgCO2_per_kg_path = df_input_params_all.merge(
    right = df_kgCO2_per_kg_path, 
    on = 'run #'
    )

# save dataframe as csv
df_kgCO2_per_kg_path.to_csv(
    os.path.join(output_folder, 'output_emissions by pathway.csv'),         
    index = False
    )

df_kgCO2_per_kg_path

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,dehydr. reactor volume (m^3),number of dehydr. reactors,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. reactor energy (unit TBD),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days),pathway,total emissions (kg CO2-eq/kg H2)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,0.073103,1,9.646712,3500,1,0,0,1,LOHC - formic acid,9.923975
1,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,0.073103,1,9.646712,3500,1,0,0,1,compressed hydrogen,1.169879
2,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,0.073103,1,9.646712,3500,1,0,0,1,liquid hydrogen,2.629718
3,FA - delivery only,1,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500,1,0,0,1,LOHC - formic acid,22.04466
4,FA - delivery only,1,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500,1,0,0,1,compressed hydrogen,0.325656
5,FA - delivery only,1,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500,1,0,0,1,liquid hydrogen,0.093998
6,FA - closed loop,2,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500,1,0,0,1,LOHC - formic acid,0.214799
7,FA - closed loop,2,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500,1,0,0,1,compressed hydrogen,0.325656
8,FA - closed loop,2,2022,1000,1.0,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500,1,0,0,1,liquid hydrogen,0.093998
9,FA - closed loop - 80 cap factor,3,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,0.073103,1,9.646712,3500,1,0,0,1,LOHC - formic acid,0.214799


#### OUTPUT: total costs ($/kg H2) and emissions (kg CO2/kg H2) by pathway and process

In [16]:
# calculate total $/kg H2 cost by pathway and process
df_usd_per_kg_proc = df_usd_per_kg.groupby(
    by = ['run #', 'pathway', 'process'])['value'].sum().reset_index()

# update column name
df_usd_per_kg_proc.rename(
    columns = {'value' : 'total levelized cost of hydrogen ($/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_usd_per_kg_proc = df_input_params_all.merge(
    right = df_usd_per_kg_proc, 
    on = 'run #'
    )

# save dataframe as csv
df_usd_per_kg_proc.to_csv(
    os.path.join(output_folder, 'output_cost by process.csv'),         
    index = False
    )

df_usd_per_kg_proc

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,number of dehydr. reactors,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. reactor energy (unit TBD),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days),pathway,process,total levelized cost of hydrogen ($/kg H2)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,preconditioning,0.005925
1,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,production,25.155667
2,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,reconditioning,6.087458
3,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,transport,0.994497
4,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,compressed hydrogen,preconditioning,2.48097
5,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,compressed hydrogen,production,0.31
6,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,compressed hydrogen,reconditioning,2.503494
7,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,compressed hydrogen,transport,0.877489
8,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,liquid hydrogen,preconditioning,4.788187
9,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,liquid hydrogen,production,0.31


In [17]:
# calculate total kg CO2/kg H2 emissions by pathway and process
df_kgCO2_per_kg_proc = df_kgCO2_per_kg.groupby(
    by = ['run #', 'pathway', 'process'])['value'].sum().reset_index()

# update column name
df_kgCO2_per_kg_proc.rename(
    columns = {'value' : 'total emissions (kg CO2-eq/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_kgCO2_per_kg_proc = df_input_params_all.merge(
    right = df_kgCO2_per_kg_proc, 
    on = 'run #'
    )

# save dataframe as csv
df_kgCO2_per_kg_proc.to_csv(
    os.path.join(output_folder, 'output_emissions by process.csv'),         
    index = False
    )

df_kgCO2_per_kg_proc

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,number of dehydr. reactors,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. reactor energy (unit TBD),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days),pathway,process,total emissions (kg CO2-eq/kg H2)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,preconditioning,0.0
1,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,production,7.699788
2,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,reconditioning,2.009389
3,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,transport,0.214799
4,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,compressed hydrogen,preconditioning,0.533237
5,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,compressed hydrogen,production,0.0
6,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,compressed hydrogen,reconditioning,0.310986
7,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,compressed hydrogen,transport,0.325656
8,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,liquid hydrogen,preconditioning,2.423692
9,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,liquid hydrogen,production,0.0


#### OUTPUT: total costs ($/kg H2) and emissions (kg CO2/kg H2) by pathway and location

In [18]:
# calculate total $/kg H2 cost by pathway and location
df_usd_per_kg_loc = df_usd_per_kg.groupby(
    by = ['run #', 'pathway', 'location'])['value'].sum().reset_index()

# update column name
df_usd_per_kg_loc.rename(
    columns = {'value' : 'total levelized cost of hydrogen ($/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_usd_per_kg_loc = df_input_params_all.merge(
    right = df_usd_per_kg_loc, 
    on = 'run #'
    )

# save dataframe as csv
df_usd_per_kg_loc.to_csv(
    os.path.join(output_folder, 'output_cost by location.csv'),         
    index = False
    )

df_usd_per_kg_loc

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,number of dehydr. reactors,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. reactor energy (unit TBD),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days),pathway,location,total levelized cost of hydrogen ($/kg H2)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,refueling station,6.087458
1,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,terminal,15.020301
2,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,truck,11.135788
3,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,compressed hydrogen,refueling station,2.503494
4,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,compressed hydrogen,terminal,2.79097
5,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,compressed hydrogen,truck,0.877489
6,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,liquid hydrogen,liquefier,4.435231
7,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,liquid hydrogen,refueling station,1.866226
8,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,liquid hydrogen,terminal,0.662956
9,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,liquid hydrogen,truck,0.309408


In [19]:
# calculate total kg CO2/kg H2 emissions by pathway and location
df_kgCO2_per_kg_loc = df_kgCO2_per_kg.groupby(
    by = ['run #', 'pathway', 'location'])['value'].sum().reset_index()

# update column name
df_kgCO2_per_kg_loc.rename(
    columns = {'value' : 'total emissions (kg CO2-eq/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_kgCO2_per_kg_loc = df_input_params_all.merge(
    right = df_kgCO2_per_kg_loc, 
    on = 'run #'
    )

# save dataframe as csv
df_kgCO2_per_kg_loc.to_csv(
    os.path.join(output_folder, 'output_emissions by location.csv'),         
    index = False
    )

df_kgCO2_per_kg_loc

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,number of dehydr. reactors,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. reactor energy (unit TBD),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days),pathway,location,total emissions (kg CO2-eq/kg H2)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,refueling station,2.009389
1,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,terminal,7.699788
2,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,truck,0.214799
3,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,compressed hydrogen,refueling station,0.310986
4,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,compressed hydrogen,terminal,0.533237
5,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,compressed hydrogen,truck,0.325656
6,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,liquid hydrogen,liquefier,2.423572
7,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,liquid hydrogen,refueling station,0.112027
8,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,liquid hydrogen,terminal,0.000121
9,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,liquid hydrogen,truck,0.093998


#### OUTPUT: total costs ($/kg H2) and emissions (kg CO2/kg H2) by pathway and function

In [20]:
# calculate total $/kg H2 cost by pathway and function
df_usd_per_kg_func = df_usd_per_kg.groupby(
    by = ['run #', 'pathway', 'function'])['value'].sum().reset_index()

# update column name
df_usd_per_kg_func.rename(
    columns = {'value' : 'total levelized cost of hydrogen ($/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_usd_per_kg_func = df_input_params_all.merge(
    right = df_usd_per_kg_func, 
    on = 'run #'
    )

# save dataframe as csv
df_usd_per_kg_func.to_csv(
    os.path.join(output_folder, 'output_cost by function.csv'),         
    index = False
    )

df_usd_per_kg_func

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,number of dehydr. reactors,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. reactor energy (unit TBD),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days),pathway,function,total levelized cost of hydrogen ($/kg H2)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,CO2 recycling,10.141291
1,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,compression,2.604754
2,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,cooling,0.184068
3,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,pumping,0.0
4,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,purchase,0.310031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,FA - closed loop - 80 cap factor,3,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,1,9.646712,3500,1,0,0,1,liquid hydrogen,pumping,1.532563
80,FA - closed loop - 80 cap factor,3,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,1,9.646712,3500,1,0,0,1,liquid hydrogen,purchase,0.31
81,FA - closed loop - 80 cap factor,3,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,1,9.646712,3500,1,0,0,1,liquid hydrogen,storage,1.163449
82,FA - closed loop - 80 cap factor,3,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,1,9.646712,3500,1,0,0,1,liquid hydrogen,trucking,0.339161


In [21]:
# calculate total kg CO2/kg H2 emissions by pathway and function
df_kgCO2_per_kg_func = df_kgCO2_per_kg.groupby(
    by = ['run #', 'pathway', 'function'])['value'].sum().reset_index()

# update column name
df_kgCO2_per_kg_func.rename(
    columns = {'value' : 'total emissions (kg CO2-eq/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_kgCO2_per_kg_func = df_input_params_all.merge(
    right = df_kgCO2_per_kg_func, 
    on = 'run #'
    )

# save dataframe as csv
df_kgCO2_per_kg_func.to_csv(
    os.path.join(output_folder, 'output_emissions by function.csv'),         
    index = False
    )

df_kgCO2_per_kg_func

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,number of dehydr. reactors,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. reactor energy (unit TBD),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days),pathway,function,total emissions (kg CO2-eq/kg H2)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,CO2 recycling,0.0
1,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,compression,0.567446
2,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,cooling,0.081693
3,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,pumping,0.0
4,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,1,9.646712,3500,1,0,0,1,LOHC - formic acid,purchase,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,FA - closed loop - 80 cap factor,3,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,1,9.646712,3500,1,0,0,1,compressed hydrogen,trucking,0.325656
60,FA - closed loop - 80 cap factor,3,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,1,9.646712,3500,1,0,0,1,liquid hydrogen,liquefaction,0.0
61,FA - closed loop - 80 cap factor,3,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,1,9.646712,3500,1,0,0,1,liquid hydrogen,pumping,0.0
62,FA - closed loop - 80 cap factor,3,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,1,9.646712,3500,1,0,0,1,liquid hydrogen,purchase,0.0


#### OUTPUT: total costs ($/kg H2) and emissions (kg CO2/kg H2) by pathway, process, and function

In [22]:
# calculate total $/kg H2 cost by pathway, process, and function
df_usd_per_kg_proc_func = df_usd_per_kg.groupby(
    by = ['run #', 'pathway', 'process', 'function']
    )['value'].sum().reset_index()

# update column name
df_usd_per_kg_proc_func.rename(
    columns = {'value' : 'total levelized cost of hydrogen ($/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_usd_per_kg_proc_func = df_input_params_all.merge(
    right = df_usd_per_kg_proc_func, 
    on = 'run #'
    )

# save dataframe as csv
df_usd_per_kg_proc_func.to_csv(
    os.path.join(output_folder, 'output_cost by process-function.csv'),         
    index = False
    )

df_usd_per_kg_proc_func

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. reactor energy (unit TBD),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days),pathway,process,function,total levelized cost of hydrogen ($/kg H2)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,9.646712,3500,1,0,0,1,LOHC - formic acid,preconditioning,pumping,0.0
1,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,9.646712,3500,1,0,0,1,LOHC - formic acid,preconditioning,storage,0.005925
2,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,9.646712,3500,1,0,0,1,LOHC - formic acid,production,CO2 recycling,10.141291
3,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,9.646712,3500,1,0,0,1,LOHC - formic acid,production,compression,0.0
4,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,9.646712,3500,1,0,0,1,LOHC - formic acid,production,pumping,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,FA - closed loop - 80 cap factor,3,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,9.646712,3500,1,0,0,1,liquid hydrogen,production,purchase,0.31
120,FA - closed loop - 80 cap factor,3,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,9.646712,3500,1,0,0,1,liquid hydrogen,reconditioning,pumping,1.504522
121,FA - closed loop - 80 cap factor,3,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,9.646712,3500,1,0,0,1,liquid hydrogen,reconditioning,storage,0.750318
122,FA - closed loop - 80 cap factor,3,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,9.646712,3500,1,0,0,1,liquid hydrogen,reconditioning,vaporization,0.056949


In [23]:
# calculate total kg CO2/kg H2 emissions by pathway, process, and function
df_kgCO2_per_kg_proc_func = df_kgCO2_per_kg.groupby(
    by = ['run #', 'pathway', 'process', 'function']
    )['value'].sum().reset_index()

# update column name
df_kgCO2_per_kg_proc_func.rename(
    columns = {'value' : 'total emissions (kg CO2-eq/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_kgCO2_per_kg_proc_func = df_input_params_all.merge(
    right = df_kgCO2_per_kg_proc_func, 
    on = 'run #'
    )

# save dataframe as csv
df_kgCO2_per_kg_proc_func.to_csv(
    os.path.join(output_folder, 'output_emissions by process-function.csv'),         
    index = False
    )

df_kgCO2_per_kg_proc_func

,scenario,run #,output dollar year,target station capacity (kg/day),station annual capacity factor,target number of stations,one-way delivery distance (mile),electricity cost ($/kWh),diesel cost ($/gallon),electricity emission factor (kg CO2/kWh),...,dehydr. catalyst amount (kg),dehydr. catalyst cost ($/kg),dehydr. catalyst lifetime (yr),dehydr. reactor energy (unit TBD),dehydr. gas/liquid separator energy (unit TBD),station LOHC storage amount (days),pathway,process,function,total emissions (kg CO2-eq/kg H2)
0,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,9.646712,3500,1,0,0,1,LOHC - formic acid,preconditioning,pumping,0.0
1,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,9.646712,3500,1,0,0,1,LOHC - formic acid,production,CO2 recycling,0.0
2,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,9.646712,3500,1,0,0,1,LOHC - formic acid,production,compression,0.0
3,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,9.646712,3500,1,0,0,1,LOHC - formic acid,production,purchase,0.0
4,baseline,0,2022,1000,1.0,10,100,0.1709,6.028,0.228,...,9.646712,3500,1,0,0,1,LOHC - formic acid,production,reaction,7.699788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,FA - closed loop - 80 cap factor,3,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,9.646712,3500,1,0,0,1,liquid hydrogen,preconditioning,liquefaction,0.0
80,FA - closed loop - 80 cap factor,3,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,9.646712,3500,1,0,0,1,liquid hydrogen,preconditioning,pumping,0.0
81,FA - closed loop - 80 cap factor,3,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,9.646712,3500,1,0,0,1,liquid hydrogen,production,purchase,0.0
82,FA - closed loop - 80 cap factor,3,2022,1000,0.8,10,100,0.1709,6.028,0.000,...,9.646712,3500,1,0,0,1,liquid hydrogen,reconditioning,pumping,0.0
